In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../..'))
from rlrom.testers import RLTester

/home/alex/rlrom_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tester = RLTester('cfg_hug.yml')


In [5]:
list_models, _ = tester.find_huggingface_models()
print(list_models[0])

ModelInfo(id='ThomasSimonini/demo-hf-CartPole-v1', author=None, sha=None, created_at=datetime.datetime(2022, 3, 2, 23, 29, 5, tzinfo=datetime.timezone.utc), last_modified=None, private=False, disabled=None, downloads=2, downloads_all_time=None, gated=None, gguf=None, inference=None, inference_provider_mapping=None, likes=0, library_name='stable-baselines3', tags=['stable-baselines3', 'CartPole-v1', 'deep-reinforcement-learning', 'reinforcement-learning', 'model-index', 'region:us'], pipeline_tag='reinforcement-learning', mask_token=None, card_data=None, widget_data=None, model_index=None, config=None, transformers_info=None, trending_score=0, siblings=None, spaces=None, safetensors=None, security_repo_status=None, xet_enabled=None)


In [ ]:
test_result = tester.run_cfg_test()


.....
